In [1]:
import pandas as pd
import numpy as np

In [2]:
## Leitura dos dados externos
country = pd.read_json('tavbase/countries.json', orient='index').reset_index()
cities = pd.read_json('tavbase/cities.json')

In [3]:
## Tratamento dos dados externos

country = country.drop(columns=['native'])
country = country[country['index'] == 'US']
country['capital'] = 'Washington' # Altera a capital de "Washington D.C." para "Washington" para ser relacionado com a cidade da base "cities"

cities = cities[cities['country'] == 'US']

paisDados = cities.groupby('country')['lat', 'lng'].mean()
paisDados = paisDados.reset_index()
paisDados = paisDados.rename(
    columns={
        'lat': 'country_lat',
        'lng': 'country_lng'
    }
)
country = country.merge(paisDados, left_on='index', right_on='country', how='left')
capital = country.merge(cities, left_on=['index','capital'], right_on=['country','name'], how='left')
capital = capital.drop(columns=['country_x','country_y','name_y'])
capital = capital.rename(columns={
    'index': 'pais_sigla',
    'name_x': 'country',
    'lat': 'capital_lat',
    'lng': 'capital_lng'
})
cities = cities.merge(capital, left_on='country', right_on='pais_sigla', how='left')

C:\Users\duduf\AppData\Local\Temp\ipykernel_23220\164353650.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  paisDados = cities.groupby('country')['lat', 'lng'].mean()


In [4]:
## Adiciona a sigla do estado às informações
data = pd.read_feather('tavbase/Sample.feather')
usStates = pd.read_feather('tavbase/estadosUS.feather')
usStates
usStates = usStates.rename(columns={
    'usps': 'estado_sigla',
    'name': 'estado'
})
data = data.merge(usStates, left_on=['State'], right_on=['estado'], how='left')
data.drop(columns=['estado'])

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,...,OPeriod,Order Date Month,sYear,sMonth,sPeriod,Ship Date Month,Delivery,Price,Benefit,estado_sigla
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,35,2016-11-01,2016,11,35,2016-11-01,3,130.98,1,KY
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,35,2016-11-01,2016,11,35,2016-11-01,3,243.98,1,KY
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,...,30,2016-06-01,2016,6,30,2016-06-01,4,7.31,1,CA
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,22,2015-10-01,2015,10,22,2015-10-01,7,191.52,0,FL
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,22,2015-10-01,2015,10,22,2015-10-01,7,11.18,1,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,...,1,2014-01-01,2014,1,1,2014-01-01,2,8.42,1,FL
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,38,2017-02-01,2017,3,39,2017-03-01,5,45.98,1,CA
9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,38,2017-02-01,2017,3,39,2017-03-01,5,129.29,1,CA
9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,38,2017-02-01,2017,3,39,2017-03-01,5,7.40,1,CA
